## 06 - Azure Cosmos DB

In this task, we will look at how to create an Azure Cosmos DB account and create a new database and collection.

1. Open the Azure portal.
1. Search for `Azure Cosmos DB` and click on it.
1. Click on the `+ Create` button to create a new Azure Cosmos DB account.
1. Select `Azure Cosmos DB for NoSQL` as the API.

   ![Create Azure Cosmos DB account](../images/06-azure-cosmos-db-apis.png)

1. Fill in the necessary details to create a new Azure Cosmos DB account.

   > Important: Cosmos DB account names must be globally unique. If you see an error message that the name is already taken, try a different name.

   ![Create Azure Cosmos DB account](../images/06-azure-cosmos-db-create.png)

1. Click `Review + Create` and then click `Create`.
1. After the account is created, open the new account in the Azure Portal.
1. Select the `Keys` blade from the `Settings` section.
1. Copy the `Primary Key` from the `Readd-write keys` section and use it in the code below.

   ![Azure Cosmos DB Keys](../images/06-azure-cosmos-db-keys.png)


### Accessing Azure Cosmos DB with Python

In this task, we will store a user comment for an example blogging platform in an Azure Cosmos DB container.
We will use the SQL API to:

1. Create the database.
1. Create a container with the blog post identifier as the partition key.
1. Add a user comment to the container for a blog post.
1. Read all the comments for a blog post.

You'll need to get the Cosmos DB Connecting String from the Azure Portal.

In [8]:
# Uncomment next line if not running in Devcontainer
# %pip install azure-cosmos

from azure.cosmos import exceptions, CosmosClient, PartitionKey
import uuid
import datetime

# Configure these for your Azure Cosmos DB Account
account_name = '<your Cosmos DB account name>'
account_key = '<your Cosmos DB account key>' # Note: a better way is to use Entra ID, but this is out of scope.

# Define the database name
database_name = 'blogdb'

# Define the container name
container_name = 'blogcomments'

In [18]:
# Using Python as azure.cosmos, Create the Cosmos DB client
url = f"https://{account_name}.documents.azure.com:443/"
client = CosmosClient(url, {'masterKey': account_key})

# Create the database if it doesn't exist
try:
    database = client.create_database(id=database_name)
except exceptions.CosmosResourceExistsError:
    database = client.get_database_client(database=database_name)

# Create the container if it doesn't exist
try:
    container = database.create_container(id=container_name, partition_key=PartitionKey(path="/blog_id"))
except exceptions.CosmosResourceExistsError:
    container = database.get_container_client(container_name)

# Display the database throughput and settings.
database_properties = database.read()
print(f"Database properties: {database_properties}")

# Display the container throughput and settings
container_properties = container.read()
print(f"Container properties: {container_properties}")


Database properties: {'id': 'blogdb', '_rid': 'XwRGAA==', '_self': 'dbs/XwRGAA==/', '_etag': '"00000f02-0000-1a00-0000-668224f50000"', '_colls': 'colls/', '_users': 'users/', '_ts': 1719805173}
Container properties: {'id': 'blogcomments', 'indexingPolicy': {'indexingMode': 'consistent', 'automatic': True, 'includedPaths': [{'path': '/*'}], 'excludedPaths': [{'path': '/"_etag"/?'}]}, 'partitionKey': {'paths': ['/blog_id'], 'kind': 'Hash', 'version': 2}, 'conflictResolutionPolicy': {'mode': 'LastWriterWins', 'conflictResolutionPath': '/_ts', 'conflictResolutionProcedure': ''}, 'geospatialConfig': {'type': 'Geography'}, '_rid': 'XwRGAJ9FeFQ=', '_ts': 1719808599, '_self': 'dbs/XwRGAA==/colls/XwRGAJ9FeFQ=/', '_etag': '"00002002-0000-1a00-0000-668232570000"', '_docs': 'docs/', '_sprocs': 'sprocs/', '_triggers': 'triggers/', '_udfs': 'udfs/', '_conflicts': 'conflicts/'}


In [21]:
# Add a new blog comment to the container
comment = {
    "id": str(uuid.uuid4()),
    "blog_id": "1",
    "user": "Daniel Scott-Raynsford",
    "comment": "I love this blog post!",
    "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
}

container.create_item(body=comment)
print(f"Added comment: {comment}")


Added comment: {'id': 'd0af6476-4b08-47b7-9c7d-b66f2a986ffc', 'blog_id': '1', 'user': 'Daniel Scott-Raynsford', 'comment': 'I love this blog post!', 'timestamp': '2024-07-01 16:36:49'}


In [23]:
# Get all the comments for the blog post with id "1" (which is the partition key)
# Don't use a cross-partition query
comments = list(container.query_items(
    query="SELECT * FROM blogcomments bc WHERE bc.blog_id = '1'",
    enable_cross_partition_query=False
))

# Display the comments
print("Comments:")
for comment in comments:
    print(comment)


Comments:
{'id': 'f2608af5-5611-4109-8eba-9ee9646098e5', 'blog_id': '1', 'user': 'Daniel Scott-Raynsford', 'comment': 'I love this blog post!', 'timestamp': '2024-07-01 16:36:44', '_rid': 'XwRGAJ9FeFQBAAAAAAAAAA==', '_self': 'dbs/XwRGAA==/colls/XwRGAJ9FeFQ=/docs/XwRGAJ9FeFQBAAAAAAAAAA==/', '_etag': '"0100aa89-0000-1a00-0000-6682325c0000"', '_attachments': 'attachments/', '_ts': 1719808604}
{'id': 'd0af6476-4b08-47b7-9c7d-b66f2a986ffc', 'blog_id': '1', 'user': 'Daniel Scott-Raynsford', 'comment': 'I love this blog post!', 'timestamp': '2024-07-01 16:36:49', '_rid': 'XwRGAJ9FeFQCAAAAAAAAAA==', '_self': 'dbs/XwRGAA==/colls/XwRGAJ9FeFQ=/docs/XwRGAJ9FeFQCAAAAAAAAAA==/', '_etag': '"0100af89-0000-1a00-0000-668232610000"', '_attachments': 'attachments/', '_ts': 1719808609}
